What this script does:
1. Load the full dataset of vacancies and merge with the SOC letters that have been estimated for each vacancy
2. Compute the un-weighted monthly stock of online job adverts (OJA) vacancies broken down by SIC
3. Compute the per-vacancy weights to align with the ONS vacancy dataset
4. Compute the adjusted per-vacancy weights after taking into account adverts with "uncertain" SIC letters
5. Save the results for future analysis

Note on the presence of adverts with "uncertain" SIC letters. 
- First of all, these adverts are assigned the median weight for that month. The same is true for job adverts with SIC codes that are not measured by the ONS vacancy survey. This affects the total stock count, which is now higher than the ONS one because the overall sum includes vacancies that were not used to compute the per-vacancy weights. However, we adjust for this, otherwise breakdowns by other characteristics might be artificially inflated.



# Imports

In [ ]:

# ------------------------ DEPENDENCIES AND FUNCTIONS ------------------------

# standard imports
from collections import Counter
from copy import deepcopy
import datetime
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from pandas.api.types import CategoricalDtype
from pathlib import Path
import pickle
import seaborn as sns
import scipy.stats as st
import sys
import statsmodels as sm
from time import time as tt
from tqdm import tqdm

# custom imports
from flow_to_stock_funcs import get_stock_breakdown, load_ons_vacancies, \
                                set_month_at_beginning, set_month_at_end, scale_weights_by_total_levels
from textkernel_load_utils import tk_params, create_tk_import_dict, read_and_append_chunks, \
                                  load_full_column, data_path, data_folder
from utils_general import nesta_colours, flatten_lol, sic_letter_to_text, print_elapsed, TaskTimer, printdf

# Add custom SIC groups
sic_letter_to_text['Z'] = 'others'
sic_letter_to_text['L_O_S'] = 'personal_and_public_services'
sic_letter_to_text['D_E'] = 'utilities'
sic_letter_to_text['M_P'] = 'educational_and_professional_activities'
sic_letter_to_text['uncertain'] = 'uncertain'

# NOTE: change to local results folder
res_folder = '/path/to/results'

timer = TaskTimer()
print('Done')


# Hardcoded parameters setup

In [ ]:
FIXED_DURATION_TH = 55
TEST_DURATION_TH = 1
CONSTANT_DUR = False
DURATION_TH = TEST_DURATION_TH if CONSTANT_DUR else FIXED_DURATION_TH
START_MONTH = '2015-03'
END_MONTH= '2019-11'
FIRST_VALID_MONTH = '2015-05'
print(f'Duration threshold is {DURATION_TH}')


# Functions, parameters and helpers

In [ ]:
#%%
def norm_series(df_series):
    """ Standardise a time series"""
    return (df_series - df_series.mean())/df_series.std()

#%%
def cap_duration(data, duration_th = 55):
    """ Durations longer than 55 days are set at 55 days"""
    data.loc[data.duration>duration_th,'duration'] = duration_th
    return data

# invert the sic letter to text mapping
sic_text_to_letter = {v: k for k,v in sic_letter_to_text.items()}


In [ ]:
# Get which TK ID value for sector corresponds to which label
def get_map_industry_label_values():
    """ """
    tmp_data = pd.read_csv(os.path.join(data_folder, tk_params.file_name_template.format(0)), 
                           compression='gzip',
                encoding = 'utf-8',usecols = ['organization_industry_label','organization_industry_value'])
    map_label2value = {}
    map_value2label = {}
    for name,g in tmp_data.groupby('organization_industry_value'):
        map_value2label[name] = g.organization_industry_label.value_counts().index.values[0]
        map_label2value[map_value2label[name]] = name
    return map_label2value, map_value2label

# create the maps
map_label2value, map_value2label = get_map_industry_label_values()

map_label2value


In [ ]:
#%%
def get_top_month(x):
    """Get month in which a vacancy is most active based on a string 
    listing all the months in which the vacancy is active
    """
    if isinstance(x,str):
        x = x.split(';')[1:]
    else:
        x = x.active_months.split(';')[1:]
    months = [t.split(': ')[0].split(' ')[1] for t in x]
    durations = [t.split(': ')[1] for t in x]
    # if multiple maxes it'll return the first one, which seems reasonable
    if len(durations):
        best_idx = np.argmax(durations)
        return months[best_idx]#, durations[best_idx]
    else:
        return 'oob'

def get_top_duration(x):
    """Get the amount of time a vacancy is active in its top month
    based on a string listing all the months in which the vacancy is active
    with respective durations
    """
    if isinstance(x,str):
        if x == 'oob':
            return 0
        x = x.split(';')[1:]
    else:
        if x.active_months == 'oob':
            return 0
        x = x.active_months.split(';')[1:]
    months = [t.split(': ')[0] for t in x]
    durations = [t.split(': ')[1] for t in x]
    # if multiple maxes it'll return the first one, which seems reasonable
    best_idx = np.argmax(durations)
    return durations[best_idx]


In [ ]:
#%%
def twin_plot(ons_data,ojv_data,xlims = [pd.to_datetime(START_MONTH + '-01'),
                                         pd.to_datetime(END_MONTH + '-01')]):
    
    """ Plot two timeseries on same axis (ONS vacancies and (un-)weighted stock)"""
    fig, ax1 = plt.subplots(figsize=(10,6))

    color = 'tab:red'
    ax1.set_xlabel('date (year-month)')
    ax1.set_ylabel('ONS vacancy stock', color=color)
    ax1.plot(ons_data, 'x-', color=color)
    ax1.tick_params(axis='y', labelcolor=color)

    ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

    color = 'tab:blue'
    ax2.set_ylabel('OJA vacancy stock', color=color)  # we already handled the x-label with ax1
    ax2.plot(ojv_data, 'o-', color=color)
    ax2.tick_params(axis='y', labelcolor=color)


    #plt.figure()
    #plt.plot(raw_jvs) #norm_series(raw_jvs))
    #plt.plot(norm_series(stock_month1)) #df_stock))
    plt.xlim(xlims[0],xlims[1])
    fig.tight_layout()
    return fig, ax1


# Load the data

## Online vacancy data

### Duration, start date, soc code and organisation name

In [ ]:
print('Usually would load Textkernel dataset')
data_df = ['main_dataset']
#%% get beginning and ending of the collection period
first_date = data_df.date.min()
last_date = data_df.date.max()



### Per-vacancy SIC code

Note:
We made an algorithm that assigned a SIC code to each vacancy based on a combination of methods. The main part of the algorithm was run separately and produced a set of candindates SIC per vacancies.


In [ ]:
# Load SIC letters column
data_df += ['final_sic_letter'] 



In [ ]:
# Composition of job adverts by SIC letter
data_df.final_sic_letter.value_counts()


## ONS data

In [ ]:
# Load ONS data on vacancies
raw_jvs_full, jvs_sic_letters = load_ons_vacancies(f"{data_path}/data")
# Change all the columns names
raw_jvs_full = raw_jvs_full.rename(columns = {t: jvs_sic_letters.loc[t] for t in jvs_sic_letters.index})

printdf(raw_jvs_full.head())


In [ ]:
# drop the columns that are not needed
raw_jvs = raw_jvs_full.drop(['D','E', 'G45', 'G46', 'G47', 'L', 'M', 'O', 'P', 'S', 'G46_47'] , axis = 1)

printdf(raw_jvs.head())


# Analysis and processing of the duration field

In [ ]:
SAVEFIGS = False


In [ ]:
# Quick analysis of duration field
full_median_duration = data_df.duration.median()
tmp = data_df.duration.dropna().value_counts().sort_index()
plt.plot(tmp.cumsum()/tmp.sum()*100)
plt.xlim([0,100])
plt.plot(DURATION_TH,tmp.cumsum()[DURATION_TH]/tmp.sum()*100,'x')
plt.xlabel('Duration value')
plt.ylabel('Proportion of jobs')
print((f'Percentage of filtered job adverts with duration within limit ({DURATION_TH} days is the threshold),'
       ' among the ones with a not null duration field:'
       f' {tmp.cumsum()[DURATION_TH]/tmp.sum()*100:.2f}%'))
if SAVEFIGS:
    plt.savefig(f"{res_folder}/cumulative_sum_of_durations.jpg")


In [ ]:
#%%
# replace "bad" duration values (that is, those that are zeros or higher than the threshold)
good_durations = (data_df.duration>0) & (data_df.duration<=duration_th)
# take the median from those durations that will not be changed
median_duration = data_df[good_durations].duration.median()
print(f'Median duration to use is {median_duration}')

if CONSTANT_DUR:
    data_df['duration_to_use'] = DURATION_TH
    print(f'Using constant duration of {DURATION_TH}')
else:
    data_df['duration_to_use'] = data_df.duration.copy()
    # replace 0s
    data_df.loc[data_df.duration_to_use==0,'duration_to_use'] = median_duration
    data_df.loc[data_df.duration_to_use>duration_th,'duration_to_use'] = duration_th
    data_df.duration_to_use = data_df.duration_to_use.fillna(median_duration)
    sns.distplot(data_df.duration_to_use)

assert(data_df.duration_to_use.isna().sum()==0)
print(f'Max duration used in the dataset is {data_df.duration_to_use.max()}')



In [ ]:
#(re-)compute end date 
'''
Note that I'm using the convention of removing 1 (even though it means I need to shift the removal day by 1 
when computing the stock) because a) it makes sense to have the expiration date rather than the removal date
and b) that is how the original expiration date is in the TK dataset
'''
data_df['end_date'] = data_df.date + pd.to_timedelta(
        data_df.duration_to_use - 1, unit='D')

# initialise weight column with 1
data_df['vacancy_weight']= 1



In [ ]:
data_df.head(5)


# Flow to stock model

In [ ]:
# get and plot un-weighted monthly stock of vacancies against ons vacancies
t0 = tt()
# new way
stock_per_month, stock_per_day, _, _ = get_stock_breakdown(
    data_df, agg_func = 'count', agg_col = 'vacancy_weight', breakdown_col = 'final_sic_letter')

print_elapsed(t0,'computing daily and monthly stock')


In [ ]:
stock_per_month.head()


In [ ]:
# Get which SIC codes are in both stocks (ONS and online job adverts)
sic_in_common = sorted(set(data_df.final_sic_letter.value_counts().index).intersection(raw_jvs.columns))
print(sic_in_common)
    

In [ ]:
# Plot each SIC letters separately: this can only be used to understand whether the trends are similar between OJA and ONS
for col in sic_in_common:
    if col == 'V':
        continue
    _ = twin_plot(1e3*raw_jvs[col], stock_per_month[col])
    tmp = np.corrcoef(raw_jvs[col].astype('float'),stock_per_month[col])[0,1]
    print((f"Time series correlation for {sic_letter_to_text[col]} is "
           f"{tmp:.3f}"))
    try:
        plt.title(sic_letter_to_text[col].capitalize())
    except:
        plt.title('others')
    if SAVEFIGS:
        plt.savefig(f"{res_folder}/raw_stock_vs_ons_sic_{col}_double_axis.jpg")


In [ ]:
# plot the time series of the non-assigned stock
plt.plot(stock_per_month['uncertain'])


In [ ]:
# Proportion of jobs without a SIC?
(data_df.final_sic_letter == 'uncertain').mean()


In [ ]:
# Get the full list of sectors and which ones are in common with the ONS data 
oja_names = sorted(stock_per_month.columns)
ons_names = [col+'_ons' for col in sorted(raw_jvs.columns) if col in oja_names]
shared_oja_names = [t for t in oja_names if t in raw_jvs.columns]
extra_oja_names = [t for t in oja_names if t not in shared_oja_names]
shared_oja_names, extra_oja_names


In [ ]:
# Keep the uncertain stock separate
stock_per_day_full = stock_per_day.copy() #[['A','T','uncertain']]
stock_per_month_full = stock_per_month.copy() #[['A','T','uncertain']]

# drop the original column
stock_per_day = stock_per_day.drop(axis = 1, labels = extra_oja_names) #['A','T','uncertain'])
stock_per_month = stock_per_month.drop(axis = 1, labels = extra_oja_names) #['A','T','uncertain'])


In [ ]:
(stock_per_day<0).sum() #THE STOCK CAN NOT BE NEGATIVE: this should be empty


# Compute post-sampling weights to align the two data sources

## ------------------- MONTHLY WEIGHTS ESTIMATES -------------------

Compute the ratio between the two stocks per month, from the ONS and from OJA

Assign an average weight to the vacancies with uncertain SIC based on their assigned month

Assigning per-vacancy weight based on an assigned month and SIC letter

Rescale the per-vacancy weight by a monthly factor to increase alignment.


### Extract needed information for each vacancy

In [ ]:
# all months of interest
all_months = pd.date_range(start= START_MONTH, end = END_MONTH, freq = 'M').map(
    set_month_at_beginning)

# initialise new dataframe - for each vacancy I want to compute the best month
# and how long it stays open during that month
data_df = data_df.assign(active_months = '')

# For each vacancy, get all the months in which it is active and the relative duration
for month in tqdm(all_months):
    tot_days = month.days_in_month
    month_begins = month
    month_ends = set_month_at_end(month)
    # extract all jobs that are active during this month
    jobs_starting_now = data_df.date.between(month_begins, month_ends)
    jobs_ending_later = ((data_df.date<month_begins) & (
                data_df.end_date>month_begins))
    valid_jobs = jobs_starting_now | jobs_ending_later

    valid_durations = (data_df[valid_jobs].end_date.map(lambda x: 
        min([x,month_ends])) - data_df[valid_jobs].date.map(lambda x: 
        max([x,month_begins]))).map(lambda x: (x.days+1)/tot_days)
                                                          
    # record the active month
    data_df.loc[valid_jobs,'active_months'] = data_df[
        valid_jobs].active_months.map(
        lambda x: x+ f';month {month.year}-{month.month:02}: ')
    # append the durations
    data_df.loc[valid_jobs,'active_months'] = data_df[
        valid_jobs].active_months + valid_durations.map(lambda x: f'{x:.3f}')



In [ ]:
#%% Add column with best month
t0 = tt()
data_df['best_month'] = data_df.active_months.map(
    get_top_month)

#turn all months to beginning of the month timestamp
data_df.best_month = pd.to_datetime(data_df.best_month).map(set_month_at_beginning)


print_elapsed(t0,'getting the best month')



In [ ]:
#%% Add column with the duration of a vacancy in its best month
t0 = tt()
data_df['best_month_duration'] = data_df.active_months.map(
    get_top_duration)
data_df.best_month_duration = data_df.best_month_duration.astype('float')
print_elapsed(t0,'getting how long vacancies are open in their best months')


### Compute the ratio between the two stocks per month, from the ONS and from OJA

In [ ]:
#%% # compute weights by months and SIC
# join ONS and OJA data
joint_stock = raw_jvs[['vacancies']+shared_oja_names].merge(stock_per_month.copy(), how = 'outer',
                            suffixes = ('_ons','_counts'),
                            left_index = True, right_index=True)

for col,ons_col in zip(shared_oja_names,ons_names):
    assert(col+'_ons'==ons_col)
    joint_stock[col+'_weight'] = joint_stock[ons_col]*1000/joint_stock[col+'_counts'].replace(0,pd.NA)

joint_stock = joint_stock[sorted(joint_stock.columns)]

joint_stock = joint_stock.replace(np.inf, 0)

for col in shared_oja_names:
    joint_stock[col+'_weight'] = joint_stock[col+'_weight'].astype('float')

# rename the columns
joint_stock = joint_stock.rename(columns = {'vacancies': 'vacancies_ons'})#, 

# replace NaN with the neutral weight (which is 1)
joint_stock = joint_stock.fillna(1)

print('Done')


#### Assign an average weight to the vacancies with uncertain SIC based on their assigned month

In [ ]:
# Add the weight for the 'uncertain' category as the median across all SIC codes
# I need to replicate them for all the categories of SIC codes that are not present in the ONS survey
joint_stock = joint_stock.assign(uncertain_weight = joint_stock[
    [col+'_weight' for col in shared_oja_names]].median(axis = 1))

joint_stock = joint_stock.assign(A_weight = joint_stock.uncertain_weight.values)

joint_stock = joint_stock.assign(T_weight = joint_stock.uncertain_weight.values)

# rename columns for consistency
for col in oja_names:
    if col not in shared_oja_names:
        joint_stock = joint_stock.rename(columns = {col: col+'_counts'})

joint_stock = joint_stock[sorted(joint_stock.columns)]

In [ ]:
joint_stock.head(5)


In [ ]:
# example weights for agriculture
joint_stock.iloc[:5].A_weight


### Merge the monthly weights with the main dataframe 
That is, Assigning per-vacancy weight based on an assigned month and SIC letter

In [ ]:
# Prepare dataframe for merging, i.e. put it in long form
weights_cols = [col for col in joint_stock.columns if 'weight' in col]
joint_stock_weights = joint_stock[weights_cols]
joint_stock_weights = joint_stock_weights.reset_index()
joint_stock_weights = pd.melt(joint_stock_weights, id_vars='month', 
                              value_vars = weights_cols,
                             value_name='vacancy_weight_adj',
                             var_name = 'sic_letter')
joint_stock_weights = joint_stock_weights.rename(columns = {'month': 'best_month'})
joint_stock_weights.sic_letter = joint_stock_weights.sic_letter.map(lambda x: x[:-7])
joint_stock_weights


In [ ]:
#%%%
# [if needed] remove weights from previous iterations with vacancies per month
if 'vacancy_weight_adj' in data_df.columns:
    print('removing old iteration of monthly weights')
    data_df = data_df.drop(axis = 1, labels = 'vacancy_weight_adj')
    #data_df = data_df[keep_columns]


#%%
# Merge vacancy weights based on sic classification and month
timer.start_task('joining new monthly weights')

small_df = None
# MERGING WEIGHTS WITHIN THE MAIN DATAFRAME
data_df = pd.merge(data_df, joint_stock_weights, 
                               left_on = ['final_sic_letter','best_month'], 
                               right_on= ['sic_letter','best_month'],
                               how = 'left')
assert(old_len_data== len(data_df))

data_df.vacancy_weight = data_df.vacancy_weight_adj
timer.end_task()

# multiply by the duration percentage
data_df['vacancy_weight_adj'] = data_df.vacancy_weight * data_df.best_month_duration



### Rescale the per-vacancy weight by a monthly factor to increase alignment

This function is used to scale the per-vacancy weights used to align the stock of online job vacancies with the stock of vacancies from the ONS survey. For more info see the docstring of the function 'scale_weights_by_total_levels'

In [ ]:

# compute the scaled weights
new_weights_df = scale_weights_by_total_levels(joint_stock_weights.rename(
    columns = {'sic_letter': 'final_sic_letter', 'vacancy_weight_adj': 'vacancy_weight'}), 
    raw_jvs, stock_per_month_full, sectors_in_common = shared_oja_names)


In [ ]:
# join up with main dataframe
timer.start_task('joining dataframe to add re-scale adjustment weights')
data_df = data_df.merge(new_weights_df[['best_month','final_sic_letter','vacancy_weight_new']], 
                        on = ['best_month','final_sic_letter'], how ='left')
timer.end_task()


In [ ]:
data_df = data_df.rename(columns = {'vacancy_weights_new': 'vacancy_weight_new'})


## Re-compute the stock of vacancies and show the results


In [ ]:
#%% get new daily stock of vacancies
timer.start_task('recomputing daily and monthly OJV stock')
new_stock_per_month, _, _ , _ = get_stock_breakdown(data_df, agg_func = 'sum', 
                               agg_col = 'vacancy_weight_new', breakdown_col = 'final_sic_letter')
timer.end_task()


In [ ]:
# Plot ONS and weighted OJA stock for each SIC letters separately. Total levels are likely to be different
for col in shared_oja_names:
    if col in ['A','T','uncertain']:
        plt.plot(new_stock_per_month[col])
        plt.title("Stock of 'uncertain' vacancies")
    else:
        plt.figure(figsize = (8,8))
        plt.plot(new_stock_per_month[col],label = 'OJV after')
        plt.plot(1000*raw_jvs[col],'--',label='ONS')
        plt.plot(stock_per_month[col],label = 'OJV before')
        plt.xlabel('Date', fontsize = 13)
        plt.ylabel('Vacancy stock', fontsize = 13)
        plt.legend(fontsize = 13)
        tmp = np.corrcoef(raw_jvs[col].astype('float'),new_stock_per_month[col])[0,1]
        print((f"Time series correlation for {sic_letter_to_text[col]} is "
               f"{tmp:.3f}"))
        plt.title(sic_letter_to_text[col])
        if SAVEFIGS:
            plt.savefig(f"{res_folder}/adjusted_stock_vs_raw_vs_ons_sic_{col}_single_axis")
    

In [ ]:
# merge the new stock per month with the un-weighted one
joint_stock = joint_stock.merge(new_stock_per_month, left_index = True, 
                                right_index = True)
joint_stock = joint_stock.rename(columns = 
                                 {col: col+'_sum' for col in new_stock_per_month.columns})
joint_stock = joint_stock[sorted(joint_stock.columns)]
joint_stock.head()


In [ ]:
# compute and show the correlation between ONS and OJA stock before (un-weighted) and after (weighted)
for col in shared_oja_names: #new_stock_per_month.columns:
    if col in ['A','T','uncertain']:
        continue
    print(f"Correlation before and after for {col}")
    joint_stock[col+'_ons'] = joint_stock[col+'_ons'].astype('float')
    print(joint_stock[[col+'_ons',col+'_counts',col+'_sum']].corr()[col+'_ons'])
    print(f"MSE before and after for {col}")
    mse_before = ((joint_stock[col+'_ons'] - joint_stock[col+'_counts']/1000)**2).sum()
    mse_after = ((joint_stock[col+'_ons'] - joint_stock[col+'_sum']/1000)**2).mean()
    print(pd.Series([mse_before,mse_after],index = [col + t for t in ['_counts','_sum']]))
    print()
    

# Save monthly weights to disk 

Do this so that we can load and join them with the main dataframe for future analysis
